## Introduction

## Neuronal Spiking Models

There are different models to describe the dynamics of neuronal spiking. In this work, we use two variants of the Integrate-and-Fire model family: the QIF model and the LIF model. The membrane potential $x$ in both models follows a specific differential equation as long as the potential remains below a threshold value $V_t$. If the membrane potential exceeds this value, it is reset to a resting value $V_r$.

**Firing conditions:**
- If $x \geq V_t$, then $x \rightarrow V_r$.

### LIF Model (Leaky Integrate-and-Fire)
The differential equation describing the membrane potential $x$ is:

$$
\frac{dx}{dt} = -x + I
$$

### QIF Model (Quadratic Integrate-and-Fire)

The differential equation describing the QIF variable $v$ is:

$$
\frac{dv}{dt} = v^2 + I
$$

This model can be equivalently transformed into the **theta model**, where the membrane potential $x$ evolves according to:

$$
\frac{dx}{dt} = 1 - \cos x + (1 + \cos x) I
$$

where

$$
v = \tan(x / 2)
$$


## Variables of Each Neuron
Each neuron in the network is associated with two variables:
- $x$: membrane potential  
- $r$: describes the output current of the neuron.

How does $r$ evolve?  
It decays exponentially:

$$
\frac{dr}{dt}= -b*r
$$

And when a spike occurs in $x$:

$$
r = r + b
$$

(You can check this paper: https://pubmed.ncbi.nlm.nih.gov/19003450/ about dynamics in the quadratic integrate and fire neuron with adaptation)


## Modeling the Synaptic Current
Neurons do not operate in isolation. They receive current from other neurons and can also generate an output current that serves as input for other neurons.

We assign the connection strength between two neurons using a coefficient. If the presynaptic neuron is neuron $j$ and the postsynaptic neuron is neuron $i$, the connection strength is represented by the matrix element $W_{ij}$ in the matrix $W$, which we call the weight matrix.

We model the input current $v$ to neuron $i$ as:

$$
v = \sum_j W_{ij} r_j
$$

where $r_j$ represents the output of neuron $j$.




- We have a network of **N** nodes, which represent the neurons.  
- There are $pqif * N$ neurons that follow the QIF dynamics and $(1 - pqif) * N$ neurons that follow the LIF dynamics.  
- Each node has two associated variables, as we saw before: `r` and `x`. These evolve according to the differential equations described earlier.  
- The different nodes are connected to each other with a probability `p`. These connections represent the synapses between neurons.  
- The connections between neurons are stored in the weight matrix `W`, where the component $W_{ij}$ refers to the connection between a postsynaptic neuron *i* and a presynaptic neuron *j*. In this work, once the weight matrix is initialized, **we do not modify it yet** (there is no learning).  
- The evolution of the system is determined by a time variable `it`, which increases step by step by a value `dt`.  
- We are interested in measuring the average of the output `r` in time windows of duration `itmax`. We call this average `rprom`.  
- We measure the variable `rprom` a total of `nloop` times.  



In [12]:
import numpy as np
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import csv

In [13]:
# Cantidad de neuronas
N = 200                                   
N2 = int(N/2)

# Probabilidad de conexion
p = 0.3

# Parámetros temporales
dt = 0.1
itmax = 200
nloop = 2000

#Parámetros de la dinámica
sigman = 1 # noise size
vt = 0.1  # threshold
b = 0.5    # bias para la salida de la neurona

np.random.seed(80)                       # para repetitividad

Evolution of the variables `r` and `x`:  
- `dynamics`: This function gives the evolution of the subthreshold membrane potential. We add a noise component of magnitude `sigman`. Note that the equation for the QIF dynamics here is not the same as the one presented before. The expression used here is obtained through a change of variables. Consequently, the spike detection condition also changes.  
- `detect`: This function detects when the membrane potential exceeds the threshold value `Vt`.  

In addition to updating the membrane potentials, the variable `r` is also updated according to the equations described earlier. A spike counter `nspike` is also included.  


In [14]:
def dynamics(x_var, r_var, I_var, nqif):
    dx = np.zeros(N)
    # LIF
    dx[nqif:] = -x_var[nqif:] + I_var[nqif:] + np.random.randn(N - nqif)*sigman
    # QIF
    dx[:nqif] = 1 - np.cos(x_var[:nqif]) + I_var[:nqif] * (1 + np.cos(x_var[:nqif])) + np.random.randn(nqif)*sigman

    dr = -b*r_var
    return dx, dr


def detect(x, xnew, rnew, nspike, nqif):
    # LIF
    ispike_lif = np.where((x[nqif:] < vt) & (xnew[nqif:] > vt))[0] + nqif
    if(len(ispike_lif) > 0):
        rnew[ispike_lif[:]] +=  b
        xnew[ispike_lif[:]] = 0
        nspike[ispike_lif[:]] += 1
    # QIF
    dpi = np.mod(np.pi - np.mod(x, 2*np.pi), 2*np.pi)  # distancia a pi
    ispike_qif = np.where((xnew[:nqif] - x[:nqif] > 0) & (xnew[:nqif] - x[:nqif] - dpi[:nqif] > 0))[0]
    if(len(ispike_qif) > 0):
        rnew[ispike_qif[:]] += b
        nspike[ispike_qif[:]] += 1
    return xnew, rnew, nspike

Here we add functions to initialize the connectivity matrix `W` and to initialize the variables of the neurons.


In [15]:
def initialize_connectivity_matrix(N, p, gsyn):
    w = sparse.random(N, N, p, data_rvs=np.random.randn).todense()
    np.fill_diagonal(w, 0)  # No autapses
    w *= gsyn / np.sqrt(p * N)
    
    for i in range(N):
        i0 = np.where(w[i, :])[1]
        if len(i0) > 0:
            av0 = np.sum(w[i, i0]) / len(i0)
            w[i, i0] -= av0
    
    return w

def initialize_neurons(N, itmax):
    x = np.random.uniform(size=N) * 2 * np.pi
    r = np.zeros(N)
    nspike = np.zeros(N)

    return x, r, nspike

We run the simulation with all the previously mentioned components.
The differential equations are integrated using a **second-order Runge-Kutta method**.

In [16]:
def run_simulation(N, nloop, itmax, nqif, x, r, nspike, w):

    rprom = np.zeros((N, nloop))
    rprom = np.matrix(rprom)

    for iloop in range(nloop):

        for it in range(itmax):
        
            v = np.matmul(w, r.T)
            v = np.squeeze(np.asarray(v))
            
            dx, dr = dynamics(x, r, v, nqif)
            xnew = x + dt * dx / 2
            rnew = r + dt * dr / 2
            dx, dr = dynamics(xnew, rnew, v, nqif)
            xnew = x + dt * dx
            rnew = r + dt * dr

            xnew, rnew, nspike = detect(x, xnew, rnew, nspike, nqif)

            x = np.copy(xnew)
            r = np.copy(rnew)

            rprom[:, iloop] += r[:, np.newaxis]

        rprom[:, iloop] /= itmax

    return rprom

In [17]:
def main():

    for pqif in [0]:


        nqif = int(N * pqif)
        gsyn = 10
        
        x, r, nspike = initialize_neurons(N, itmax)
        w = initialize_connectivity_matrix(N, p, gsyn)
        
        rprom = run_simulation(N, nloop, itmax, nqif, x, r, nspike, w)

        
if __name__ == "__main__":
    main()